In [1]:
from scipy import spatial
import pandas as pd

df = pd.DataFrame(data={'c1': [1, 2, 2], 'c2': [3, 4, 1]})

f1 = df.iloc[0, :]
f2 = df.iloc[1, :]

# compute the cosine similarity between the first 2 rows
cosine_sim = 1 - spatial.distance.cosine(f1, f2)
print(cosine_sim)

0.9899494936611665


In [2]:
import numpy as np

df = pd.DataFrame(data={'user': [1, 1, 2, 2], 'rating': [3, 4, 1, 2]})

def normalize_ratings(df, rating_col="rating", user_col="user"):
    groups = df.groupby(user_col)[rating_col]
    # computes group-wise mean/std and auto broadcasts to individual groups
    mean = groups.transform(np.mean)
    std = groups.transform(np.std)
    return (df[rating_col] - mean) / std

df["rating_normalized"] = normalize_ratings(df)
df.head()

,user,rating,rating_normalized
0,1,3,-0.707107
1,1,4,0.707107
2,2,1,-0.707107
3,2,2,0.707107


In [3]:
import numpy as np

df = pd.DataFrame(data={'user': [1, 1, 2, 2], 'rating': [10, 10, 10, 10], 't': pd.to_datetime(["2019-01-01", "2019-01-02", "2019-01-03", "2019-01-04"])})

def cumsum_days(s, duration='D'):
    return s.diff().astype('timedelta64[%s]' % duration).fillna(0).cumsum().values

def decay_ratings(df, decay=1, rating_col="rating", time_col="t"):
    weight = np.exp(-cumsum_days(df[time_col]) * decay)
    return df[rating_col] * weight

half_life_t = 1
decay = np.log(2) / half_life_t
df["rating_decayed"] = decay_ratings(df, decay=decay)

df.head()

,user,rating,t,rating_decayed
0,1,10,2019-01-01,10.00
1,1,10,2019-01-02,5.00
2,2,10,2019-01-03,2.50
3,2,10,2019-01-04,1.25


In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 156 kB/s 
     |████████████████████████████████| 198 kB 29.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=9bdbeac1772bb8760b54ee9a2d297c094b95f8e9ae4cfcc1d2a88db34815761a
  Stored in directory: /Users/ckoerner/Library/Caches/pip/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark
You should consider upgrading via the '/Users/ckoerner/.local/share/virtualenvs/packt-repo-M2qY5kM-/bin/python -m pip install --upgrade pip' command.


In [6]:
import pyspark
from pyspark.ml.recommendation import ALS
sc = pyspark.SparkContext('local[*]')

n_iter = 10
rank = 10
l2_reg = 1

als = ALS() \
    .setMaxIter(n_iter) \
    .setRank(rank) \
    .setRegParam(l2_reg)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/05 17:54:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
model = als.fit(train_data)


NameError: name 'train_data' is not defined